In [1]:
import torch
import json
import numpy as np
import torch.nn as nn
from torch.nn import functional as F
from tokenizers import ByteLevelBPETokenizer, trainers, pre_tokenizers, decoders, Tokenizer

from arch import LGGPT, SingleHead, GetTopTokens
from alpha import AlphaZeroTrainer
from embed import EmbeddingTables

In [2]:
# tokenizer = ByteLevelBPETokenizer(vocab=None, merges=None)
# tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()
# tokenizer.decoder = decoders.ByteLevel()

# # Get vocab and vocab length
# with open("new_tokenizer.json", 'r') as f:
#     json_content = f.read()

# parsed_json = json.loads(json_content)
# vocab = list(parsed_json["model"]["vocab"].keys())

tokenizer = Tokenizer.from_file("new_tokenizer.json")
tokenizer.decoder = decoders.ByteLevel()
vocab = tokenizer.get_vocab()

vocab_size = len(vocab)
print(vocab_size)

n_embd = 21
n_head = 3
dropout = 0.1
focus_len = 256
head_size = n_embd // n_head

gpt = LGGPT(vocab_size, n_embd, n_head, dropout, focus_len)
embeds = EmbeddingTables(vocab_size=vocab_size, n_embd=n_embd, focus_len=focus_len)
single_head = SingleHead(n_embd, head_size, dropout, focus_len)
trainer = AlphaZeroTrainer(model=gpt, embeds=embeds, tokenizer=tokenizer, single_head=single_head, vocab=vocab, lr=0.001)
single_head = SingleHead(n_embd, head_size, dropout, focus_len)

30000


In [3]:
# sentence = "The quick brown fox jumped over the lazy dog"
# # sentence_tokens = tokenizer.encode(sentence).ids  # Convert to token IDs
# # sentence_tensor = torch.tensor(sentence_tokens).unsqueeze(0).unsqueeze(-1)  # Add batch and feature dimensions

# # Training step
# loss = trainer.train_step(sentence, n_sims=5)
# print(f"Training loss: {loss}")

In [4]:
sentence = "The quick brown fox jumped over the lazy dog"
# sentence_tokens = tokenizer.encode(sentence).ids  # Convert to token IDs
# sentence_tensor = torch.tensor(sentence_tokens).unsqueeze(0).unsqueeze(-1)  # Add batch and feature dimensions

# Training step
loss = trainer.train_step(sentence, n_sims=1)

sentence tokens:  [462, 4794, 26347, 20930, 13983, 757, 906, 6247, 77, 4310]
data:  tensor([    2,   462,  4794, 26347, 20930, 13983,   757,   906,  6247,    77,
         4310,     3])
data shape:  torch.Size([12])
index size:  torch.Size([256])
x shape:  torch.Size([256, 21])
attended shape:  torch.Size([256, 21])
attended:  tensor([[-0.5806, -2.4734, -0.2571,  ...,  2.7967, -0.5955, -0.1159],
        [ 0.9965,  0.1544, -1.6804,  ...,  1.6156, -0.7192,  0.0615],
        [ 0.2554, -0.1855, -0.3438,  ...,  1.9990, -0.9675,  0.7281],
        ...,
        [-0.0273, -0.8210,  1.1588,  ..., -0.3338, -1.7046, -0.3155],
        [-1.0899, -1.4467,  1.2505,  ...,  1.8580, -0.2941,  0.8575],
        [ 1.3017,  0.1894,  1.1948,  ..., -0.0077, -0.3689, -0.0892]],
       grad_fn=<NativeLayerNormBackward0>)
[1, 1, 1, 1]
policy_probs: [3.2711669e-05 4.0513132e-05 3.2647658e-05 ... 2.9849527e-05 3.7365779e-05
 3.3938712e-05], shape: (570000,)


/Users/bennyrose/Little-Guy/lg/alpha.py:115: RuntimeWarning: invalid value encountered in divide
  policy_target = visit_counts / visit_counts.sum()


RuntimeError: shape '[10]' is invalid for input of size 1

In [9]:
del dropout, sentence, single_head, tokenizer, trainer, vocab
import gc
gc.collect()

31